# Un brin d’algorithmique avec XPath

## Avant-propos

Documentation : https://www.w3.org/TR/xpath20/

Le langage XPath dans sa version 2.0 offre la possibilité de recourir à des structures pour construire un algorithme :
- opérations arithmétiques
- comparaisons
- expressions logiques
- structures conditionnelles
- structures itératives

**Remarque :** d’autres structures et expressions existent.

## Fragment XML

```xml
<constellations>
    <constellation origine="Lacaille">
    <nom fr="La Carène">Carina</nom>
        <etendues>
            <etendue unite="deg2">494.184</etendue>
            <etendue unite="%">1.20</etendue>
        </etendues>
    </constellation>
    <constellation origine="Ptolémée">
    <nom fr="Le Corbeau">Corvus</nom>
        <etendues>
            <etendue unite="deg2">183.801</etendue>
            <etendue unite="%">0.45</etendue>
        </etendues>
    </constellation>
</constellations>
```

## Opérations arithmétiques

### Addition

Quelle surface du ciel visible représentent les deux premières constellations ?

```xml
//constellation[1]//etendue[@unite = '%'] + //constellation[2]//etendue[@unite = '%']
```

### Soustraction

Après soustraction de l’étendue du nœud courant, que représentent les étendues des autres constellations ?

```xml
sum(//etendue[@unite = 'deg2']) - etendue[@unite = 'deg2']
```

### Multiplication

Quelle serait la surface d’une constellation deux fois plus grande que Carina ?

```xml
//constellation[nom = 'Carina']//etendue[@unite = 'deg2'] * 2
```

### Division

L’étendue du nœud courant représente quel pourcentage des étendues totales ?

```xml
etendue[@unite = '%'] div sum(//etendue[@unite = '%'])
```

### Modulo

Grâce à la division euclidienne, serait-il possible de répartir équitablement entre les constellations la surface du ciel visible qu’elles occupent ?

```xml
sum(//etendue[@unite = '%']) mod count(//constellation)
```

## Comparaisons

### Inférieur à

Pour juger si une valeur est inférieure à une autre. Opérateurs : `<` `lt` `<=` `le`

```xml
<!-- true -->
//constellation[nom = 'Carina'] lt //constellation[nom = 'Corvus']
```

### Supérieur à

Pour juger si une valeur est supérieure à une autre. Opérateurs : `>` `gt` `>=` `ge`

```xml
etendue[@unite = '%'] gt 1
```

### Égalité

Détermine si deux valeurs sont identiques ou différentes. Opérateurs : `=` `eq` `!=` `ne`

```xml
nom eq 'Carina'
```

### Référence

Détermine si deux valeurs font référence au même nœud.

```xml
<!-- true -->
/constellations/constellation[nom = 'Corvus'] is /constellations/constellation[nom/@fr = 'Le Corbeau']
```

## Expressions logiques

### Conjonction

Permet de combiner des expressions entre elles pour ajouter des contraintes dans un prédicat ou pour effectuer un test logique.

```xml
<!-- Carina -->
//constellation[@origine = 'Lacaille' and //etendue[@unite = 'deg2'] > '400']/nom

<!-- false -->
//constellation[@origine = 'Lacaille'] and //constellation[@origine = 'Tycho Brahe']
```

### Disjonction

Permet de combiner des expressions entre elles pour augmenter la sélection de nœuds ou pour effectuer un test logique.

```xml
<!-- Carina, Corvus -->
//constellation[@origine = 'Lacaille' or //etendue[@unite = 'deg2'] > '400']/nom

<!-- true -->
//constellation[@origine = 'Lacaille'] or //constellation[@origine = 'Tycho Brahe']
```

## Structures conditionnelles

Mots-clés `if` `then` `else`

Les structures conditionnelles permettent de tester une expression et de lui associer un comportement en cas de succès ou d’échec.

```xml
<!-- Lacaille -->
if (
    //constellation[nom = 'Carina']//etendue[@unite = '%'] gt
    //constellation[nom = 'Corvus']//etendue[@unite = '%']
)
then 'Lacaille'
else 'Ptolémée'
```

## Structures itératives

Mots-clés `for` `return`

Les structures itératives permettent de parcourir une liste de nœuds.

```xml
<!-- Carina, Corvus -->
for $a in //constellation
return $a/nom
```

```xml
<!-- 494.184, 183.801 -->
for $a in //etendue
return $a[@unite = 'deg2']
```

```xml
<!-- Lacaille : Carina ; Ptolémée : Corvus -->
for $a in distinct-values(constellation/@origine)
return (
    distinct-values(constellation/@origine[. = $a]),
    constellation[@origine = $a]/nom
)
```

Elle peuvent aussi se combiner avec d’autres structures, comme les conditionnelles :

```xml
<!-- Carina -->
for $a in //constellation[@origine = 'Lacaille']
return
    if ($a//etendue[@unite = 'deg2'] gt '450')
    then $a/nom
    else $a//etendue[@unite = 'deg2']
```